In [1]:
import pinocchio as pin
from pinocchio import casadi as cpin
import casadi
import numpy as np
import example_robot_data as robex
#import matplotlib.pyplot as plt; plt.ion()
from pinocchio.visualize import GepettoVisualizer
from utils.meshcat_viewer_wrapper import  MeshcatVisualizer, colors

In [2]:
robot = robex.load('talos')
model = robot.model
cmodel = cpin.Model(robot.model)
data = model.createData()
cdata = cmodel.createData()

q0 = robot.q0
v0 = np.zeros(robot.nv)
a0 = np.zeros(robot.nv)

In [3]:
""" from pinocchio.visualize import GepettoVisualizer

### HYPER PARAMETERS
# Hyperparameters defining the optimal control problem.
T = 10
DT = 0.05

### LOAD AND DISPLAY SOLO
# Load the robot model from example robot data and display it if possible in Gepetto-viewer
viz_debug = GepettoVisualizer(robot.model,robot.collision_model,robot.visual_model)
viz_debug.initViewer()
viz_debug.loadViewerModel()

viz_debug.display(robot.q0) """

' from pinocchio.visualize import GepettoVisualizer\n\n### HYPER PARAMETERS\n# Hyperparameters defining the optimal control problem.\nT = 10\nDT = 0.05\n\n### LOAD AND DISPLAY SOLO\n# Load the robot model from example robot data and display it if possible in Gepetto-viewer\nviz_debug = GepettoVisualizer(robot.model,robot.collision_model,robot.visual_model)\nviz_debug.initViewer()\nviz_debug.loadViewerModel()\n\nviz_debug.display(robot.q0) '

In [4]:
viz = MeshcatVisualizer(robot)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [5]:
viz.display(robot.q0)

In [25]:
class KineModel:
    def __init__ (self, cmodel):
        self.cmodel = cmodel
        self.cdata = cdata = cmodel.createData()
        self.nq, self.nv = cmodel.nq, cmodel.nv
        self.nx = self.nq+ self.nv

        cq = casadi.SX.sym('cq', self.nq, 1)
        cv = casadi.SX.sym('cv', self.nv, 1)
        ca = casadi.SX.sym('ca', self.nv, 1)
        cx = casadi.vertcat(cq, cv)

        self.IDX_LF = IDX_LF = cmodel.getFrameId('leg_left_6_link')
        self.IDX_RF = IDX_RF = cmodel.getFrameId('leg_right_6_link')

        self.com_position = casadi.Function('com', [cq], [cpin.centerOfMass(cmodel, cdata, cq)] )

        cpin.forwardKinematics(cmodel, cdata, cq, cv, ca)
        cpin.updateFramePlacements(cmodel, cdata)

        self.lf_position = casadi.Function('lf', [cq], [cdata.oMf[IDX_LF].translation])
        self.rf_position = casadi.Function('rf', [cq], [cdata.oMf[IDX_RF].translation])
        self.rf_rotation = casadi.Function('rf', [cq], [cdata.oMf[IDX_RF].rotation])

    def cost(self, cq):
        com_pos = self.com_position(cq)
        return com_pos.T @ com_pos 

    def foot_constraint(self, q):
        r = self.rf_rotation(q)
        p = self.rf_position(q)
        return cpin.log6(cpin.SE3(r, p)).vector
        opti.subject_to(casadi.sumsqr(cpin.SE3(r,p).translation) == 0) 

        

In [26]:
kine_model = KineModel(cmodel)

In [28]:
cq = casadi.SX.sym('cq', robot.nq, 1)
cv = casadi.SX.sym('cv', robot.nv, 1)
ca = casadi.SX.sym('ca', robot.nv, 1)

""" cq = casadi.MX.sym('cq', robot.nq, 1)
cv = casadi.MX.sym('cv', robot.nv, 1)
ca = casadi.MX.sym('ca', robot.nv, 1) """

#casadi.sumsqr(kine_model.foot_constraint(cq))

#cpin.integrate(cmodel, cq, cv)

" cq = casadi.MX.sym('cq', robot.nq, 1)\ncv = casadi.MX.sym('cv', robot.nv, 1)\nca = casadi.MX.sym('ca', robot.nv, 1) "

In [29]:
ballId = 'world/ball'
viz.display(q0)
viz.addSphere(ballId, 0.1, color=colors.red)
pose = pin.SE3(kine_model.rf_rotation(q0).full(), kine_model.rf_position(q0).full() )
viz.applyConfiguration(ballId, pose)

In [30]:
opti = casadi.Opti()

dxs = opti.variable(kine_model.nx, 1)

opti.minimize(kine_model.cost(dxs[: kine_model.nq]))
opti.subject_to(casadi.sumsqr(kine_model.foot_constraint(dxs[: kine_model.nq])) == 0)


ArgumentError: Python argument types in
    SE3.__init__(SE3, MX, MX)
did not match C++ signature:
    __init__(boost::python::api::object, pinocchio::SE3Tpl<double, 0> clone)
    __init__(_object* self, pinocchio::SE3Tpl<casadi::Matrix<casadi::SXElem>, 0> clone)
    __init__(_object* self, Eigen::Matrix<casadi::Matrix<casadi::SXElem>, 4, 4, 0, 4, 4> array)
    __init__(_object* self, int int)
    __init__(_object* self, Eigen::Quaternion<casadi::Matrix<casadi::SXElem>, 0> quat, Eigen::Matrix<casadi::Matrix<casadi::SXElem>, 3, 1, 0, 3, 1> translation)
    __init__(_object* self, Eigen::Matrix<casadi::Matrix<casadi::SXElem>, 3, 3, 0, 3, 3> rotation, Eigen::Matrix<casadi::Matrix<casadi::SXElem>, 3, 1, 0, 3, 1> translation)
    __init__(_object* self)

In [157]:
opti.solver('ipopt')

opti.solve()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.11.9, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      777

Total number of variables............................:       77
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

OptiSol(Opti {
  instance #18
  #variables: 1 (nx = 77)
  #parameters: 0 (np = 0)
  #constraints: 0 (ng = 0)
  CasADi solver allocated.
  CasADi solver was called: Solve_Succeeded
})

In [158]:
x_sol = opti.value(dxs)
q_sol = x_sol[: kine_model.nq]
v_sol = x_sol[kine_model.nq:]

In [159]:
viz.display(q_sol)